In [7]:
import pymysql
import requests
import pandas as pd
import joblib
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from rapidfuzz import process, fuzz

# 환경 변수 로드 (없을 경우 기본값 설정)
load_dotenv()

DB_HOST = os.getenv("DB_HOST", "localhost")
DB_USER = os.getenv("DB_USER", "root")
DB_PASSWORD = os.getenv("DB_PASSWORD", "")
DB_NAME = os.getenv("DB_NAME", "financial_db")

def get_db_connection():
    """ 데이터베이스 연결을 생성하는 함수 """
    try:
        conn = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            cursorclass=pymysql.cursors.DictCursor
        )
        print("데이터베이스 연결 성공!")
        return conn
    except pymysql.MySQLError as e:
        print(f"데이터베이스 연결 실패: {e}")
        return None

# DB 연결 테스트 실행
conn = get_db_connection()
if conn:
    conn.close()


데이터베이스 연결 성공!


In [8]:
load_dotenv()

True

In [9]:
def get_db_connection():
    return pymysql.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME"),
        cursorclass=pymysql.cursors.DictCursor
    )

In [10]:
api_key = os.getenv("OPEN_API_KEY_sec")
llm = ChatOpenAI(model_name="gpt-4o-mini", api_key=api_key)

/var/folders/pb/nb0b8jkd25n9nqcy75ftffym0000gn/T/ipykernel_25247/3693358954.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", api_key=api_key)


In [11]:
def load_corp_mapping():
    conn = get_db_connection()
    with conn.cursor() as cursor:
        query = "SELECT corp_code, stock_name FROM company_list"
        cursor.execute(query)
        result = cursor.fetchall()
    
    corp_mapping = {row["stock_name"]: row["corp_code"] for row in result}
    conn.close()
    return corp_mapping

corp_mapping = load_corp_mapping()

def get_corp_code(stock_name):
    return corp_mapping.get(stock_name, None)  # 매칭되지 않으면 None 반환

In [12]:
# query = "SELECT corp_code, concept_id, a FROM company_list WHERE corp_code = '00111847'"
# conn = get_db_connection()

# with conn.cursor() as cursor:
#     cursor.execute(query)
#     company_profile = cursor.fetchone()

In [13]:
def get_company_profile(corp_code):
    query = "SELECT corp_code, stock_name a FROM company_list WHERE corp_code = %s"

    conn = get_db_connection()
    company_profile = None
    
    try:
        with conn.cursor() as cursor:
            cursor.execute(query, (corp_code,))
            company_profile = cursor.fetchone()
    except Exception as e:
        print(f"회사 프로필 조회 중 오류 발생: {e}")
    finally:
        conn.close()
        
    return company_profile

In [ ]:
def get_financial_data(corp_code):
    """
    특정 기업의 재무 데이터를 조회하여 DataFrame으로 변환하는 함수.
    
    Args:
        corp_code (str): 기업의 코드
        conn: MySQL 데이터베이스 연결 객체

    Returns:
        dict: 손익계산서(IS), 재무상태표(BS), 포괄손익계산서(CIS) 데이터가 포함된 사전.
    """
    queries = {
        "is": "SELECT corp_code, concept_id, 2023amount, 2022amount, 2021amount FROM single_is WHERE corp_code = %s",
        "bs": "SELECT corp_code, concept_id, 2023amount, 2022amount, 2021amount FROM single_bs WHERE corp_code = %s",
        "cis": "SELECT corp_code, concept_id, 2023amount, 2022amount, 2021amount FROM single_cis WHERE corp_code = %s"
    }

    conn = get_db_connection()
    financial_data = {}

    try:
        with conn.cursor() as cursor:
            for key, query in queries.items():
                cursor.execute(query, (corp_code,))
                result = cursor.fetchall()

                # 데이터가 없으면 빈 DataFrame 생성
                if not result:
                    financial_data[key] = pd.DataFrame(columns=["corp_code", "concept_id", "2023amount", "2022amount", "2021amount"])
                else:
                    financial_data[key] = pd.DataFrame(result, columns=["corp_code", "concept_id", "2023amount", "2022amount", "2021amount"])
        return financial_data
    
    except Exception as e:
        print(f"❌ 데이터 조회 중 오류 발생: {e}")
        return None

In [97]:
def get_economic_indicators():
    FRED_API_KEY = "8cb84788382477974f9d9e4b69a76ccd"
    FRED_BASE_URL = "https://api.stlouisfed.org/fred/series/observations"
    FRED_INDICATORS = {
        "Exchange Rate (USD/KRW)": "DEXKOUS",
        "WTI Crude Oil Price": "DCOILWTICO"
    }

    start_year = 2020
    end_year = 2024

    economic_data = {"year": [], "indicator": [], "value": []}

    for indicator_name, series_id in FRED_INDICATORS.items():
        url = f"{FRED_BASE_URL}?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json"
        
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()["observations"]
            
            for entry in data:
                year = int(entry["date"][:4])
                if start_year <= year <= end_year:
                    economic_data["year"].append(year)
                    economic_data["indicator"].append(indicator_name)
                    economic_data["value"].append(float(entry["value"]) if entry["value"] != "." else None)
        except requests.exceptions.RequestException as e:
            print(f"⚠️ {indicator_name} 데이터 요청 실패: {e}")

    df_econ = pd.DataFrame(economic_data)
    df_econ["year"] = pd.to_datetime(df_econ["year"], format="%Y").dt.year  # 'year'를 datetime으로 변환
    df_econ_avg = df_econ.groupby(["year", "indicator"])["value"].mean().reset_index()
    
    df_econ_pivot = df_econ_avg.pivot(index="year", columns="indicator", values="value").reset_index()
    return df_econ_pivot

economic_data = get_economic_indicators()

In [ ]:
def preprocess_financial_data(financial_data):
    """
    재무 데이터를 전처리하여 모델 입력 형식으로 변환.
    
    Args:
        financial_data (dict): IS, BS, CIS 데이터가 포함된 딕셔너리
    
    Returns:
        DataFrame: 모델이 학습한 Feature 형식으로 변환된 데이터
    """
    # 1. 데이터 병합 (3개년 데이터 포함)
    df_all = pd.concat([
        financial_data.get("is", pd.DataFrame(columns=["corp_code", "concept_id", "2023amount", "2022amount", "2021amount"])),
        financial_data.get("bs", pd.DataFrame(columns=["corp_code", "concept_id", "2023amount", "2022amount", "2021amount"])),
        financial_data.get("cis", pd.DataFrame(columns=["corp_code", "concept_id", "2023amount", "2022amount", "2021amount"]))
    ], ignore_index=True)

    # 2. 데이터 타입 변환 (숫자로 변환)
    for year in ["2023amount", "2022amount", "2021amount"]:
        df_all[year] = pd.to_numeric(df_all[year], errors="coerce")

    # 3. 데이터 Pivot 변환 (연도별로 구분)
    df_pivot = df_all.pivot_table(index="corp_code", columns="concept_id", values=["2023amount", "2022amount", "2021amount"], aggfunc="sum").reset_index()
    
    # 4. 사용할 Feature 리스트 (연도별 변수 포함)
    selected_features = [
        "ifrs-full_Assets",
        "dart_OperatingIncomeLoss",
        "ifrs-full_CurrentAssets",
        "ifrs-full_CurrentLiabilities",
        "ifrs-full_Equity",
        "ifrs-full_IssuedCapital",
        "ifrs-full_Liabilities",
        "ifrs-full_NoncurrentAssets",
        "ifrs-full_NoncurrentLiabilities",
        "ifrs-full_ProfitLoss",
        "ifrs-full_PropertyPlantAndEquipment",
        "ifrs-full_GrossProfit",
        "ifrs-full_CostOfSales"
    ]

    outlook_features = [
        "ifrs-full_Revenue", # 매출액
        "ifrs-full_Assets",
        "dart_OperatingIncomeLoss",
        "ifrs-full_CurrentAssets",
        "ifrs-full_CurrentLiabilities",
        "ifrs-full_Equity",
        "ifrs-full_IssuedCapital",
        "ifrs-full_Liabilities",
        "ifrs-full_NoncurrentAssets",
        "ifrs-full_NoncurrentLiabilities",
        "ifrs-full_ProfitLoss",
        "ifrs-full_PropertyPlantAndEquipment",
        "ifrs-full_GrossProfit",
        "ifrs-full_CostOfSales"
    ]

    # 5. 연도별 Feature 선택 (모델은 2023년 데이터 사용)
    print("5단계")
    feature_columns = ["corp_code"] + [("2023amount", feature) for feature in selected_features]
    outlook_columns = ["corp_code"] + [("2023amount", feature) for feature in outlook_features]

    # 6. Feature 선택 및 NaN 값 0으로 채우기
    print("6단계")
    outlook_df = df_pivot.reindex(columns=outlook_columns, fill_value=0)
    print(outlook_df)
    X_new = df_pivot.reindex(columns=feature_columns, fill_value=0)
    print(X_new)
    print("7단계")
    # 7. 컬럼명 변환 (다층 인덱스를 단일 컬럼으로 변환)
    X_new.columns = ["corp_code"] + selected_features
    print("너도 ㅇ[러냐?")
    outlook_df.columns = ["corp_code"] + outlook_features
    print("너ㅏㅓ냐?")
    result = []
    print("8단계")
    
    result.append(X_new)
    result.append(outlook_df)
    
    print("=" * 10)
    print(result[0])
    print("=" * 10)
    print(result[1])
    return result


In [99]:
# financial_data = get_financial_data("00111847")

In [100]:
# preprocess_financial_data(financial_data)

In [101]:
def format_korean_currency(value):
    trillion = value // 1_0000_0000_0000
    billion = (value % 1_0000_0000_0000) // 1_0000_0000

    if trillion > 0 and billion > 0:
        return f"{trillion}조 {billion}억"
    elif trillion > 0:
        return f"{trillion}조"
    else:
        return f"{billion}억"

In [102]:
MODEL_PATH = os.path.join("예측모델", "linear_regression_model.pkl")
model = joblib.load(MODEL_PATH)

# 예측 수행채ㅜ
def predict_revenue(X_new):
    predicted_revenue = model.predict(X_new)
    return predicted_revenue[0]

In [103]:
def determine_outlook(operating_margin, debt_ratio):
    """
    매출 성장률, 영업이익률, 부채비율을 기반으로 기업 전망을 결정
    """
    if operating_margin > 10 and debt_ratio < 150:
        return "긍정 📈"
    elif operating_margin < 5 or debt_ratio > 200:
        return "부정 📉"
    else:
        return "불투명 ⚖️"

In [104]:
# def determine_outlook(sales_growth, operating_margin, debt_ratio):
#     """
#     매출 성장률, 영업이익률, 부채비율을 기반으로 기업 전망을 결정
#     """
#     if sales_growth > 5 and operating_margin > 10 and debt_ratio < 150:
#         return "긍정 📈"
#     elif sales_growth < 0 or operating_margin < 5 or debt_ratio > 200:
#         return "부정 📉"
#     else:
#         return "불투명 ⚖️"

In [105]:
def get_candidate_companies(company_name: str):
    """
    회사명을 기반으로 가장 유사한 회사 목록을 반환
    """
    candidates = process.extract(company_name, corp_mapping.keys(), scorer=fuzz.WRatio, limit=5)
    return [c[0] for c in candidates if c[1] > 70]  # 70점 이상만 반환

def get_valid_company(extracted_company: str) -> str:
    """
    추출된 기업명을 DB에서 검색하여 데이터가 있으면 그대로 사용하고,
    없으면 유사한 기업명을 자동 추천하고, 필요시 사용자 입력을 받음.
    """
    corp_code = get_corp_code(extracted_company)
    
    # 정확한 이름이 DB에 있으면 그대로 반환
    if corp_code:
        return extracted_company
    
    # DB에 없는 경우 유사한 기업명 추천
    candidates = get_candidate_companies(extracted_company)
    
    if candidates:
        best_match = candidates[0]  # 가장 유사한 기업명 선택
        print(f"🔍 입력하신 '{extracted_company}'와 가장 유사한 기업: {best_match}")
        
        # 유사도가 높으면 자동 선택
        if process.extractOne(extracted_company, [best_match], scorer=fuzz.WRatio)[1] > 85:
            print(f"'{best_match}'를 자동 선택합니다.")
            return best_match
        
        # 유사도 낮으면 사용자 입력 요청
        print(f"📌 유사한 기업 목록: {', '.join(candidates)}")
        new_company = input("올바른 기업명을 입력해주세요 (자동 추천을 원하면 Enter): ").strip()
        
        return new_company if new_company else best_match  # 사용자가 입력하지 않으면 자동 추천
    else:
        print(f"🚨 '{extracted_company}'과 유사한 기업을 찾을 수 없습니다.")
        return extracted_company

In [106]:
import json

def extract_company_and_year(user_input):
    """
    사용자의 질문에서 기업명과 연도를 추출하고 corp_code를 매칭하는 함수
    """
    extraction_prompt = f"""
    사용자의 질문에서 기업명과 연도를 정확하게 추출하세요.
    질문: "{user_input}"
    
    형식:
    {{
        "기업명": ["기업1", "기업2"],
        "연도": "연도 (없으면 최신 연도 2024로 설정)"
    }}
    """

    # LLM을 활용해 기업명과 연도 추출 (LLM 연동 필요)
    extraction_result = llm.invoke(extraction_prompt).content.strip()

    try:
        extracted_data = json.loads(extraction_result)
        extracted_names = extracted_data.get("기업명", [])
        if not isinstance(extracted_names, list):
            extracted_names = [extracted_names]

        year = int(extracted_data.get("연도", 2024))  # 연도가 없으면 2024 기본값
    except (json.JSONDecodeError, ValueError):
        return "❌ 기업명 또는 연도를 정확히 추출할 수 없습니다.", None

    # corp_code 매칭
    matched_companies = []
    for name in extracted_names:
        valid_company = get_valid_company(name)
        corp_code = get_corp_code(valid_company)

        if corp_code:
            matched_companies.append((valid_company, corp_code))  # (기업명, corp_code) 튜플 저장
        else:
            matched_companies.append((valid_company, None))  # 매칭 실패
            print(f"🚨 '{name}'과 일치하는 기업을 찾을 수 없음")

    return matched_companies, year

In [107]:
# def company_forecast(stock_name, year):
#     corp_code = get_corp_code(stock_name)
#     if not corp_code:
#         return f"❌ '{stock_name}'의 corp_code를 찾을 수 없습니다."

#     financials = get_financial_data(corp_code)

#     # ✅ Debugging: Check financial_data structure
#     print("🔍 Debug financial_data:", financials)
#     print("🔍 Type of financial_data:", type(financials))

#     if not isinstance(financials, dict):
#         print("❌ Error: financial_data should be a dictionary but received:", type(financials))
#         return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 찾을 수 없습니다."

#     if financials is None or all(df.empty for df in financials.values()):
#         return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 찾을 수 없습니다."

#     # 예측 매출 계산
#     X_new = preprocess_financial_data(financials)
#     predicted_revenue = predict_revenue(X_new)
#     formatted_revenue = f"{predicted_revenue:,.0f}"  # 예측 매출액을 한국 원화 포맷으로 변환

#     return f"📢 {stock_name}({corp_code})의 {year}년 예상 매출: {formatted_revenue} 원"


In [108]:
def company_forecast(stock_name, year):
    """
    기업의 미래 전망을 예측하는 함수.
    
    Args:
        stock_name (str): 기업명
        year (int): 예측 연도
        conn: MySQL 데이터베이스 연결 객체
    
    Returns:
        str: 예측 결과 리포트
    """
    corp_code = get_corp_code(stock_name)
    if not corp_code:
        return f"❌ '{stock_name}'의 corp_code를 찾을 수 없습니다."

    # ✅ 1. 재무 데이터 조회
    financials = get_financial_data(corp_code)

    # ✅ 2. 데이터 확인 (디버깅용)
    print("🔍 Debug financial_data:", financials)
    
    if financials is None or all(df.empty for df in financials.values()):
        return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 찾을 수 없습니다."

    # ✅ 3. 데이터 전처리
    print("시작 전")
    X_new = preprocess_financial_data(financials)[0]
    print("X_new", X_new)
    outlook_df = preprocess_financial_data(financials)[1]
    print("outlook", outlook_df)

    # # ✅ 4. 모델 로드
    # MODEL_PATH = os.path.join("예측모델", "linear_regression_model.pkl")
    # model = joblib.load(MODEL_PATH)

    # ✅ 5. 예측 결과
    # 예측 전에 corp_code 컬럼 제거
    X_new = X_new.drop(columns=["corp_code"], errors="ignore")

    formatted_revenue = f"{format_korean_currency(predict_revenue(X_new))}"  # 예측 매출액 변환

    # ✅ 7. 경제 지표 가져오기
    economic_data = get_economic_indicators()
    economic_data_year = economic_data[economic_data["year"] == year]
    exchange_rate = economic_data_year["Exchange Rate (USD/KRW)"].values[0]
    oil_price = economic_data_year["WTI Crude Oil Price"].values[0]

    # ✅ 8. 기업 개요 및 최근 사업 동향
    company_profile = get_company_profile(corp_code)  
    business_summary = llm.invoke(f"{company_profile} 기반으로 최근 사업 동향을 3줄 요약해 주세요.").content.strip()

    # ✅ 9. 재무 지표 계산
    try:
        operating_margin = (outlook_df["dart_OperatingIncomeLoss"].values[0] /
                            (outlook_df["ifrs-full_Revenue"].values[0])) * 100
        debt_ratio = (X_new["ifrs-full_Liabilities"].values[0] / X_new["ifrs-full_Equity"].values[0]) * 100
        liquidity_ratio = (X_new["ifrs-full_CurrentAssets"].values[0] / X_new["ifrs-full_CurrentLiabilities"].values[0]) * 100
    except KeyError as e:
        print(f"❌ KeyError: {e} - 해당 지표가 없음")
        return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 충분히 확보하지 못했습니다."
    
    # load the determine_outlook func

    outlook = determine_outlook(operating_margin, debt_ratio)
    
    report = f"""
    📢 **{stock_name}({corp_code}) {year}년 전망 보고서**

    📑 **미래 전망**: {outlook}
    
    💰 **예측 매출액**: {formatted_revenue} 원

    📊 **예측에 영향을 주는 요인**
    - 🌍 **거시경제지표**
      - 환율 변동성 (평균): {exchange_rate} 원/USD
      - 원유 가격 (평균): {oil_price} USD/배럴

    - 🏢 **기업 동향**
      {business_summary}

    - 📈 **기업 재무 상태**
      - 영업이익률: {operating_margin:.2f}%
      - 부채비율: {debt_ratio:.2f}%
      - 유동비율: {liquidity_ratio:.2f}%

    💡 **전망 요약**
    - {stock_name}의 {year}년 매출은 **{formatted_revenue} 원**으로 예측됨.
    - 거시경제 지표(환율, 유가) 및 기업 동향에 따라 실적 변동 가능성이 있음.
    - 투자 시 **재무 안정성과 시장 트렌드를 고려**하는 것이 중요함.
    """

    return report


In [92]:
# def company_forecast(stock_name, year):
#     corp_code = get_corp_code(stock_name)
#     if not corp_code:
#         return f"❌ '{stock_name}'의 corp_code를 찾을 수 없습니다."

#     financials = get_financial_data(corp_code)

#     # ✅ Debugging: Check financial_data structure
#     print("🔍 Debug financial_data:", financials)
#     print("🔍 Type of financial_data:", type(financials))

#     if not isinstance(financials, dict):
#         print("❌ Error: financial_data should be a dictionary but received:", type(financials))
#         return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 찾을 수 없습니다."

#     if financials is None or all(df.empty for df in financials.values()):
#         return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 찾을 수 없습니다."

#     # 예측 매출 계산
#     X_new = preprocess_financial_data(financials)
#     predicted_revenue = predict_revenue(X_new)
#     formatted_revenue = f"{predicted_revenue:,.0f}"  # 예측 매출액을 한국 원화 포맷으로 변환

#     # 경제 지표 가져오기
#     economic_data = get_economic_indicators()
#     economic_data_year = economic_data[economic_data["year"] == year]
#     exchange_rate = economic_data_year["Exchange Rate (USD/KRW)"].values[0]
#     oil_price = economic_data_year["WTI Crude Oil Price"].values[0]

#     # 기업 동향
#     company_profile = get_company_profile(corp_code)  # DB에서 회사 개요 가져오기
#     business_summary = llm.invoke(f"{company_profile} 기반으로 최근 사업 동향을 3줄 요약해 주세요.").content.strip()

#     # 재무 지표 계산
#     operating_margin = (financials["dart_OperatingIncomeLoss"].values[0] /
#                         (financials["ifrs-full_GrossProfit"].values[0] + financials["ifrs-full_CostOfSales"].values[0])) * 100
#     debt_ratio = (financials["ifrs-full_Liabilities"].values[0] / financials["ifrs-full_Equity"].values[0]) * 100
#     liquidity_ratio = (financials["ifrs-full_CurrentAssets"].values[0] / financials["ifrs-full_CurrentLiabilities"].values[0]) * 100

#     # # 미래 전망 평가
#     # if sales_growth > 5 and operating_margin > 10 and debt_ratio < 150:
#     #     outlook = "긍정 📈"
#     # elif sales_growth < 0 or operating_margin < 5 or debt_ratio > 200:
#     #     outlook = "부정 📉"
#     # else:
#     #     outlook = "불투명 ⚖️"

#     # 최종 보고서 출력
#     report = f"""
#     📢 **{stock_name}({corp_code}) {year}년 전망 보고서**


#     💰 **예측 매출액**: {formatted_revenue} 원

#     📊 **예측에 영향을 주는 요인**
#     - 🌍 **거시경제지표**
#       - 환율 변동성 (평균): {exchange_rate} 원/USD
#       - 원유 가격 (평균): {oil_price} USD/배럴

#     - 🏢 **기업 동향**
#       {business_summary}

#     - 📈 **기업 재무 상태**
#       - 영업이익률: {operating_margin:.2f}%
#       - 부채비율: {debt_ratio:.2f}%
#       - 유동비율: {liquidity_ratio:.2f}%

#     💡 **전망 요약**
#     - {stock_name}의 {year}년 매출은 **{formatted_revenue} 원**으로 예측됨.
#     - 거시경제 지표(환율, 유가) 및 기업 동향에 따라 실적 변동 가능성이 있음.
#     - 투자 시 **재무 안정성과 시장 트렌드를 고려**하는 것이 중요함.
#     """

#     return report

In [93]:
# def company_forecast(stock_name, year):
#     corp_code = get_corp_code(stock_name)
#     if not corp_code:
#         return f"❌ '{stock_name}'의 corp_code를 찾을 수 없습니다."

#     financials = get_financial_data(corp_code)

#     # ✅ Debugging: Check financial_data structure
#     print("🔍 Debug financial_data:", financials)
#     print("🔍 Type of financial_data:", type(financials))

#     if not isinstance(financials, dict):
#         print("❌ Error: financial_data should be a dictionary but received:", type(financials))
#         return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 찾을 수 없습니다."

#     if financials is None or all(df.empty for df in financials.values()):
#         return f"❌ {stock_name}({corp_code})의 {year}년 재무 데이터를 찾을 수 없습니다."

#     # 예측 매출 계산
#     X_new = preprocess_financial_data(financials)
#     predicted_revenue = predict_revenue(X_new)
#     formatted_revenue = f"{predicted_revenue:,.0f}"  # 예측 매출액을 한국 원화 포맷으로 변환

#     # 경제 지표 가져오기
#     economic_data = get_economic_indicators()
#     economic_data_year = economic_data[economic_data["year"] == year]
#     exchange_rate = economic_data_year["Exchange Rate (USD/KRW)"].values[0]
#     oil_price = economic_data_year["WTI Crude Oil Price"].values[0]

#     # 기업 동향
#     company_profile = get_company_profile(corp_code)  # DB에서 회사 개요 가져오기
#     business_summary = llm.invoke(f"{company_profile} 기반으로 최근 사업 동향을 3줄 요약해 주세요.").content.strip()

#     # 재무 지표 계산
#     sales_growth = financials["Sales_Growth"].values[0]
#     operating_margin = (financials["dart_OperatingIncomeLoss"].values[0] /
#                         (financials["ifrs-full_GrossProfit"].values[0] + financials["ifrs-full_CostOfSales"].values[0])) * 100
#     debt_ratio = (financials["ifrs-full_Liabilities"].values[0] / financials["ifrs-full_Equity"].values[0]) * 100
#     liquidity_ratio = (financials["ifrs-full_CurrentAssets"].values[0] / financials["ifrs-full_CurrentLiabilities"].values[0]) * 100

#     # 미래 전망 평가
#     if sales_growth > 5 and operating_margin > 10 and debt_ratio < 150:
#         outlook = "긍정 📈"
#     elif sales_growth < 0 or operating_margin < 5 or debt_ratio > 200:
#         outlook = "부정 📉"
#     else:
#         outlook = "불투명 ⚖️"

#     # 최종 보고서 출력
#     report = f"""
#     📢 **{stock_name}({corp_code}) {year}년 전망 보고서**

#     🔎 **미래 전망**: {outlook}

#     💰 **예측 매출액**: {formatted_revenue} 원

#     📊 **예측에 영향을 주는 요인**
#     - 🌍 **거시경제지표**
#       - 환율 변동성 (평균): {exchange_rate} 원/USD
#       - 원유 가격 (평균): {oil_price} USD/배럴

#     - 🏢 **기업 동향**
#       {business_summary}

#     - 📈 **기업 재무 상태**
#       - 매출 성장률: {sales_growth}%
#       - 영업이익률: {operating_margin:.2f}%
#       - 부채비율: {debt_ratio:.2f}%
#       - 유동비율: {liquidity_ratio:.2f}%

#     💡 **전망 요약**
#     - {stock_name}의 {year}년 매출은 **{formatted_revenue} 원**으로 예측됨.
#     - 거시경제 지표(환율, 유가) 및 기업 동향에 따라 실적 변동 가능성이 있음.
#     - 투자 시 **재무 안정성과 시장 트렌드를 고려**하는 것이 중요함.
#     """

#     return report


In [109]:
def interactive_forecast(user_input: str):
    """
    사용자 입력을 받아 기업명과 연도를 추출하고, 해당 기업의 미래 전망 보고서를 생성
    """
    print("\n📢 기업명을 입력하면 해당 연도의 전망을 예측합니다. (종료하려면 'exit' 입력)\n")


    # ✅ 기업명 & 연도 추출
    matched_companies, year = extract_company_and_year(user_input)
    
    if matched_companies == "❌ 기업명 또는 연도를 정확히 추출할 수 없습니다.":
        print(matched_companies)
    
    for company_name, corp_code in matched_companies:
        if corp_code:
            response = company_forecast(company_name, year)
            print(response)
        else:
            print(f"❌ '{company_name}'의 corp_code를 찾을 수 없습니다.\n")


if __name__ == "__main__":
    query = "현대위아 2024년 전망 예측"
    interactive_forecast(query)


📢 기업명을 입력하면 해당 연도의 전망을 예측합니다. (종료하려면 'exit' 입력)

스똬트 :  {'is': Empty DataFrame
Columns: [corp_code, concept_id, 2023amount, 2022amount, 2021amount]
Index: [], 'bs':    corp_code                                         concept_id  \
0   00106623                            ifrs-full_CurrentAssets   
1   00106623                   ifrs-full_CashAndCashEquivalents   
2   00106623              ifrs-full_OtherCurrentFinancialAssets   
3   00106623                  ifrs-full_CurrentTradeReceivables   
4   00106623               ifrs-full_CurrentLoansAndReceivables   
5   00106623                       ifrs-full_OtherCurrentAssets   
6   00106623                         ifrs-full_CurrentTaxAssets   
7   00106623                              ifrs-full_Inventories   
8   00106623                    ifrs-full_CurrentContractAssets   
9   00106623  ifrs-full_NoncurrentAssetsOrDisposalGroupsClas...   
10  00106623                         ifrs-full_NoncurrentAssets   
11  00106623           ifrs-fu